# Script for Sweeping the Magnet on Dilution Refrigerator

## This block of code must be run- it imports all the relevant libraries and packages

In [ ]:
from threaded_sweep import Sweep1D, Sweep0D, Sweep2D
from daq_driver import Daq, DaqAOChannel, DaqAIChannel
import nidaqmx
import time
import numpy as np
import qcodes as qc
from util import init_database, set_magnet_ramp_ranges
from tracking import *
from qcodes.dataset.measurements import Measurement
from qcodes.dataset.database import initialise_or_create_database_at
from qcodes.dataset.data_export import get_data_by_id
from qcodes.instrument_drivers.Lakeshore.Model_372 import Model_372
from qcodes.instrument_drivers.american_magnetics.AMI430 import AMI430
from qcodes.instrument_drivers.stanford_research.SR860 import SR860

## Initialize the magnet here

In [ ]:
magnet = AMI430("Magnet", address="10.155.94.127", port=7180, has_current_rating=False)

### Set magnet parameter's here, like ramp rate

In [ ]:
ramp_rate_units = 'minutes'
field_ramp_rate = .08
magnet.field_ramp_limit.set(field_ramp_rate)
magnet.ramp_rate_units.set(ramp_rate_units)

### Alternatively, use the ramp ranges functionality, demonstrated below

In [ ]:
# Pass a list, formatted as [(<ramp stage>, <field ramp rate>, <max applicable current>),  . . .]
ramp_settings = 
[(1, 0.05, 10), (2, 0.08, 20), (3, 0.20, 80)]
set_magnet_ramp_ranges(magnet, ramp_settings)

### The next blocks are specific to various instruments - you only need to run the blocks for the instruments you wish to control/get data from.

### If you want to use the Lakeshore temperature controller, run this to initialize it

In [ ]:
ls = Model_372('lakeshore_372', 'TCPIP::10.155.95.127::7777::SOCKET')
ls.ch01.enabled(True)
ls.ch02.enabled(True)
ls.ch03.enabled(True)
ls.ch05.enabled(True)
ls.ch06.enabled(True)
for ch in ls.channels:
    print(f'Temperature of {ch.short_name} ({"on" if ch.enabled() else "off"}): {ch.temperature()} {ch.units()}')

### Initialize the DAQ here

In [ ]:
daq = Daq("Dev1", "Daq")

## Create your sweep object here! 
### The arguments for the 1D sweep shown here are, in order, the parameter to be swept, the starting point, ending point, step value, delay between steps, flag to sweep back and forth, whether to save data, and plot data, and whether to plot the parameters against time or the sweeping parameter
### The only required arguments are the parameter, start, stop, and step values

In [ ]:
sweep = Sweep1D(magnet.field, 0, 5, 0.05, inter_delay = 0.5, bidirectional = False, save_data = True, 
            plot_data = True, x_axis_time = 0, plot_bin=5, instrument=magnet)

## Tell the sweep which other parameters to track during the sweep. Here, we have added the temperature and a couple DAQ channels

In [ ]:
sweep.follow_param(daq.ai0.voltage, daq.ai1.voltage, ls.ch01.temperature)

## Now, we initialize/create the database we are saving to (if you want to save data)
## You can run this code between each run to give the data identifiers through *exp_name* and *sample_name*

In [ ]:
try:
    # Make sure database_name and the path are set to the correct values!
    database_name = "temp.db"
    exp_name = "Example experiment- Magnetic field dependence"
    sample_name = "Example sample- Sweep field_Gain=10^6_T=10mK_Vg=3V_measure I12_float others"
    init_database(database_name, exp_name, sample_name, s)
except:
    print("Error opening database")

## Now, we can start the sweep! This will automatically start data collection, plotting, saving, etc

In [ ]:
%matplotlib qt
sweep.start()

### To stop the sweep, run the stop command below. To restart it, you can run the start command from above again.

In [ ]:
sweep.stop()

In [ ]:
sweep.resume()